In [2]:
import requests
from kafka import KafkaProducer
import json

def send_plane_to_kafka(topic, url, fields={}):

    # Kafka configuration
    kafka_config = {
        'bootstrap_servers': 'kafka1:9092',  # Update with your Kafka broker
    }

    # Initialize Kafka Producer
    producer = KafkaProducer(
        bootstrap_servers=kafka_config['bootstrap_servers'],
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

    # Fetch data from the api    
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        states = data.get("states", [])
        
        # Publish each entry to Kafka
       
        for state in states:
            output = {}
            for key in fields.keys():
                output[key] = state[fields[key]]
            # output["icao24"] = state[0]  # Id of the plane  (ICAO24)
            # output["origin_country"] = state[2]
            # output["velocity"] = state[9]
            # output["timestamp"] = state[3]  # Timestamp de la dernière position
            # output["latitude"] = state[6]  # Latitude de l'avion
            # output["longitude"] = state[5]  # Longitude de l'avion
            producer.send(topic, value=output)
        # Ensure all messages are sent
        producer.flush()
        print(f"Sent {len(states)} records.")
        print(f"Exemple : {states[0]}")
    else:
        print(f"Failed to fetch data: {response.status_code}")

In [3]:
from datetime import datetime, timedelta
fields = {
    "icao24":0,
    "callsign":1,
    "origin_country":2,
    "velocity":9,
    "timestamp":3,
    "latitude":6,
    "longitude":5
}

api_url = "https://opensky-network.org/api/states/all?lamin=30.0&lamax=60.0&lomin=-10&lomax=10.0"

# date initiale
initial_date_str = "2025-01-25 16:00:00"
# durée de récupération (minutes)
step = 60

date_format = "%Y-%m-%d %H:%M:%S"
initial_date = datetime.strptime(initial_date_str, date_format)
end_date = initial_date + timedelta(minutes=step)

start = initial_date.strftime(date_format)
end = end_date.strftime(date_format)

In [5]:
import time 
for i in range(10):
    print(f"{start}-{end}")        
    send_plane_to_kafka("planes", api_url, fields)
    time.sleep(40)
    initial_date = end_date
    end_date = initial_date + timedelta(minutes=step)
    start = initial_date.strftime(date_format)
    end = end_date.strftime(date_format)

2025-01-25 21:00:00-2025-01-25 22:00:00
Sent 1983 records.
Exemple : ['4b1818', 'SWR3HY  ', 'Switzerland', 1742467113, 1742467117, 8.5575, 47.4543, None, True, 0, 185.62, None, None, None, '1000', False, 0]
2025-01-25 22:00:00-2025-01-25 23:00:00
Sent 1985 records.
Exemple : ['4b1818', 'SWR3HY  ', 'Switzerland', 1742467113, 1742467117, 8.5575, 47.4543, None, True, 0, 185.62, None, None, None, '1000', False, 0]
2025-01-25 23:00:00-2025-01-26 00:00:00
Sent 1990 records.
Exemple : ['4b1818', 'SWR3HY  ', 'Switzerland', 1742467113, 1742467117, 8.5575, 47.4543, None, True, 0, 185.62, None, None, None, '1000', False, 0]
2025-01-26 00:00:00-2025-01-26 01:00:00
Sent 1993 records.
Exemple : ['4b1818', 'SWR3HY  ', 'Switzerland', 1742467113, 1742467117, 8.5575, 47.4543, None, True, 0, 185.62, None, None, None, '1000', False, 0]
2025-01-26 01:00:00-2025-01-26 02:00:00
Sent 2003 records.
Exemple : ['4b1818', 'SWR3HY  ', 'Switzerland', 1742467113, 1742467117, 8.5575, 47.4543, None, True, 0, 185.62, N

ChunkedEncodingError: Response ended prematurely

In [3]:
import requests
response = requests.get("https://opensky-network.org/api/states/all?lamin=40.0&lamax=52.0&lomin=5&lomax=10.0")

# Vérifier si la requête a réussi
if response.status_code == 200:
    data = response.json()  # Convertir la réponse en dictionnaire
    
    # Vérifier si "states" existe dans les données
    if "states" in data and data["states"]:
        for state in data["states"]:
            icao24 = state[0]  # Identifiant unique de l'avion (ICAO24)
            origin_country = state[2]
            velocity = state[9]
            timestamp = state[3]  # Timestamp de la dernière position
            latitude = state[6]  # Latitude de l'avion
            longitude = state[5]  # Longitude de l'avion

            # Afficher uniquement si latitude et longitude sont disponibles
            if latitude is not None and longitude is not None:
                print(f"ID: {icao24}, Timestamp: {timestamp}, Latitude: {latitude}, Longitude: {longitude}, Velocity : {velocity}, Origin : {origin_country}")
    else:
        print("Aucune donnée d'avion disponible.")
else:
    print("Erreur :", response.text)

ID: 4b1819, Timestamp: 1742454200, Latitude: 47.4559, Longitude: 8.5649, Velocity : 2.31, Origin : Switzerland
ID: 39de4c, Timestamp: 1742454249, Latitude: 46.9132, Longitude: 9.7358, Velocity : 228.27, Origin : France
ID: 4b1804, Timestamp: 1742454248, Latitude: 45.4209, Longitude: 6.0884, Velocity : 220.45, Origin : Switzerland
ID: 4b1802, Timestamp: 1742454248, Latitude: 51.4119, Longitude: 9.7546, Velocity : 230.3, Origin : Switzerland
ID: 4b181a, Timestamp: 1742454249, Latitude: 48.0637, Longitude: 7.2134, Velocity : 191.8, Origin : Switzerland
ID: 39de46, Timestamp: 1742454248, Latitude: 45.6294, Longitude: 9.5937, Velocity : 232.21, Origin : France
ID: 39de48, Timestamp: 1742454248, Latitude: 41.9883, Longitude: 7.7829, Velocity : 253.64, Origin : France
ID: 39de47, Timestamp: 1742454248, Latitude: 41.0186, Longitude: 8.3119, Velocity : 246.69, Origin : France
ID: 3c6667, Timestamp: 1742454249, Latitude: 49.8878, Longitude: 8.3467, Velocity : 133.81, Origin : Germany
ID: 3c664e,

In [10]:
import requests
import json

api_url = "https://opensky-network.org/api/states/all?lamin=30.0&lamax=60.0&lomin=-10&lomax=10.0"
response = requests.get(api_url)

if response.status_code == 200:
    data = response.json()["states"]
    print(len(data[3]))
    print(json.dumps(data, indent=2))  # Voir la structure des données
else:
    print("Erreur :", response.status_code)

17
[
  [
    "4b1816",
    "SWR2VM  ",
    "Switzerland",
    1742310540,
    1742310540,
    6.1694,
    47.7952,
    11582.4,
    false,
    232.25,
    301.97,
    0,
    null,
    11612.88,
    "3014",
    false,
    0
  ],
  [
    "4b1818",
    "SWR7GP  ",
    "Switzerland",
    1742310540,
    1742310540,
    7.0895,
    48.1857,
    7871.46,
    false,
    201.08,
    153.24,
    -13.33,
    null,
    7978.14,
    "2036",
    false,
    0
  ],
  [
    "39de4b",
    "TVF60KM ",
    "France",
    1742310539,
    1742310539,
    0.378,
    45.5324,
    11285.22,
    false,
    223.35,
    208.93,
    0.33,
    null,
    11285.22,
    "7635",
    false,
    0
  ],
  [
    "4b1812",
    "SWR57H  ",
    "Switzerland",
    1742310539,
    1742310539,
    9.0173,
    48.3886,
    5882.64,
    false,
    198.17,
    217.41,
    -11.05,
    null,
    5996.94,
    "1000",
    false,
    0
  ],
  [
    "408121",
    "VIR129R ",
    "United Kingdom",
    1742310540,
    1742310540,
    -0.63